In [33]:
import torch
import torch.nn.functional as F
target_masks = torch.randint(0,2,(3,1,480,480)).to(torch.float)
print(target_masks)
def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B,-1, window_size, window_size, C)
    #window_mapping = torch.arange(1 * (H // window_size) * (W // window_size)).view(1, H // window_size, W // window_size).expand(1, H, W)
    window_id = torch.arange(((H // window_size) * (W // window_size))).to(windows).view(1,1,H // window_size, W // window_size)
    window_id = F.interpolate(window_id,(H,W),mode='nearest').repeat(3,1,1,1)
    return windows,window_id

tensor([[[[0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
           1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
           0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1.,
           0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
           1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0.,
           0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
           0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
           0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
           0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
           0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
           0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
           0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
           1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0

In [45]:
target_sw = F.interpolate(target_masks, (120,120), mode='nearest')
pad_mask = torch.ones_like(target_sw)
target_sw = F.pad(target_sw,(4,4,4,4))
pad_mask = F.pad(pad_mask,(4,4,4,4))

torch.set_printoptions(threshold=torch.inf)
target_sw,ids = window_partition(target_sw.permute(0,2,3,1),8)
print(ids.shape)
o = torch.zeros(3,256,128,128).scatter_(dim=1,index=ids.long(),src=torch.ones_like(ids)).bool()
print(o[1,1]==0)
#pad_mask = window_partition(pad_mask.permute(0,2,3,1),8)

#print(target_sw.shape)
#print(target_sw[1,0,:,:,0])
#print(pad_mask[1,0,:,:,0])


torch.Size([3, 1, 128, 128])
tensor([[ True,  True,  True,  True,  True,  True,  True,  True, False, False,
         False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, 

In [18]:
# target_sw = F.pad(target_masks,(4,4,4,4))
# target_sw = F.interpolate(target_sw, (480,480), mode='nearest')
# target_sw = window_partition(target_sw.permute(0,2,3,1),8)

# print(target_sw.shape)
output= torch.randn((3,256,8,8,256))
print(output.shape)
print(output[1,0,:,:,0])

torch.Size([3, 256, 8, 8, 256])
tensor([[-2.1405, -0.6353, -0.2875,  0.7367,  1.3780,  1.0777, -0.5676, -0.0498],
        [ 1.2158,  2.3690,  1.0572,  1.0414,  0.3643,  1.4266,  1.7110, -1.7678],
        [-0.7083,  0.2535, -0.4775,  0.6346, -2.0046, -0.3314,  0.8741,  1.7376],
        [-0.3057,  0.8972, -0.8657, -1.9547, -0.1596,  0.1541,  0.9721,  0.3177],
        [-0.7095,  0.9830,  0.6127,  0.2760, -0.7823,  0.1920, -1.3184, -2.4377],
        [-1.3588,  1.7778, -1.0375,  1.3633,  0.3741, -0.2687,  0.1206,  0.9891],
        [ 0.4393,  1.6443,  0.0237,  0.2749,  1.2746, -0.1031, -0.2328,  1.1462],
        [-0.3678,  0.9315, -0.1873,  0.9935, -0.6077, -0.6863, -0.4736, -0.3097]])


In [19]:
A = torch.randn((3,1,480,480))
A_sw = F.avg_pool2d(A,4,4)#.flatten(2)
print(A_sw.shape)
print(A_sw[1,0,0:4,0:4].mean())
#A_sw = F.pad(A_sw,(4,4,4,4),mode=)#.flatten(2)
A_sw = F.avg_pool2d(A_sw,8,8,4,count_include_pad=False)#.flatten(2)
print(A_sw[1,0,0,0])

print(A_sw.shape)
print()

torch.Size([3, 1, 120, 120])
tensor(-0.0076)
tensor(-0.0076)
torch.Size([3, 1, 16, 16])



In [20]:
import numpy as np
#np.set_printoptions(threshold=np.inf)
torch.set_printoptions(threshold=3)

def _get_src_permutation_idx(self, indices):
    # permute predictions following indices
    batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
    src_idx = torch.cat([src for (src, _) in indices])
    return batch_idx, src_idx
A_sw = A_sw.flatten(2)
_,ind = torch.topk(A_sw,256)
print(ind.squeeze(1))
D = torch.tensor(0).to(A_sw)
D_num = 0
T = 0.5
batch_idx = torch.cat([torch.full_like(src, i) for i, (src) in enumerate(ind)])
# print(batch_idx)
# print((batch_idx,ind))
# print(output.shape)
# print(batch_idx.shape)
# print()
# print(target_sw[batch_idx,ind.squeeze(1)][0,0,:,:,0])
# print(target_sw[0,148,:,:,0])
select_mask = output[batch_idx,ind.squeeze(1)]
select_mask = select_mask.permute(0,1,4,2,3).flatten(3) #3,10,8,8,256 -> 3,10,256,64

select_pad_mask = pad_mask[batch_idx,ind.squeeze(1)]
select_pad_mask = select_pad_mask.permute(0,1,4,2,3).flatten(3) #3,10,8,8,1 -> 3,10,1,64

select_tgt = target_sw[batch_idx,ind.squeeze(1)]
print(select_tgt[1,0,:,:,0])
select_tgt = select_tgt.permute(0,1,4,2,3).flatten(3) #3,10,8,8,1 -> 3,10,1,64
print(select_tgt[1,0,0,])
contgt = ((select_tgt==select_tgt.permute(0,1,3,2))).long()#.to(src_masks[0])
print(contgt.shape)
print(contgt[1,0,:,:])
pos = contgt - torch.eye(64).to(contgt)#.to(src_masks[0])
neg = (contgt == 0).to(contgt)
print(pos[1,0,:,:])
print(neg[1,0,:,:])

sim = F.cosine_similarity(select_mask.unsqueeze(3),select_mask.unsqueeze(4),dim=2)
print(sim.shape)
pos_c = torch.exp((sim*pos)/T).sum(dim=-1)
neg_c = torch.exp((sim*neg)/T).sum(dim=-1)

con = -torch.log(pos_c/(pos_c+neg_c))
print(con.shape)
print(select_pad_mask.squeeze(-2).long().shape)
#con = con*select_pad_mask.squeeze(-2)
#print(con.shape)
print(con[torch.nonzero(con,as_tuple=True)].shape)
print(con[torch.nonzero(con*select_pad_mask.squeeze(-2),as_tuple=True)].shape)

con = con[torch.nonzero(select_pad_mask.squeeze(-2),as_tuple=True)]
print(con.shape)
con =con.mean()

D = D + con
#for b in range(3):
# for i in ind:
#     curmask = output[i].permute(2,0,1).flatten(1)
#     curtgt = target_sw[i].permute(2,0,1).flatten(1)
#     curpm = pad_mask[i].permute(2,0,1).flatten(1)
#     contgt = ((curtgt==curtgt.t())).long()#.to(src_masks[0])
#     pos = contgt - torch.eye(64).to(contgt)#.to(src_masks[0])
#     neg = contgt == 0
#     sim = F.cosine_similarity(curmask.unsqueeze(1),curmask.unsqueeze(2),dim=0)
#     #print(sim.shape)
#     pos_c = torch.exp((sim*pos)/T).sum(dim=1)
#     neg_c = torch.exp((sim*neg)/T).sum(dim=1)
#     con = -torch.log(pos_c/(pos_c+neg_c))[curpm.long()].mean()
#     D = D + con
#     #D = D + F.smooth_l1_loss(sim,contgt)
#     D_num = D_num +1
#     #print(D_num)
#     D = D / D_num
# print(D)

tensor([[  0, 246, 255,  ..., 176, 160, 240],
        [ 37, 240, 223,  ...,  74, 153,  87],
        [242,  16,  26,  ...,  15, 245, 160]])


IndexError: index 96 is out of bounds for dimension 1 with size 1